In [2]:
import pandas as pd

filename = "results/eval-extrinsic-20"

df_aggregated = pd.read_csv(f"{filename}.csv").sort_values("factual", ascending=False)
df_sums = pd.read_json(f"{filename}-summaries.json")

df_aggregated

,model,factual,non_factual,unknown,failed,ents_per_sum,sum_with_extrinsic,rouge1,rouge2,rougeL
0,fbs_oracle,0.70,0.30,0.0,2,3.444444,0.80,0.463874,0.236571,0.363230
1,baseline,0.50,0.50,0.0,0,3.500000,0.95,0.460633,0.227857,0.371273
2,corrector,0.45,0.55,0.0,0,3.500000,0.95,0.448691,0.216526,0.360403


In [4]:
df_sums.head()

,fbs_oracle_summary,fbs_oracle_is_non_factual,fbs_oracle_is_factual,fbs_oracle_has_unknown,fbs_oracle_has_failed,fbs_oracle_n_extrinsic,baseline_summary,baseline_is_non_factual,baseline_is_factual,baseline_has_unknown,baseline_has_failed,baseline_n_extrinsic,corrector_summary,corrector_is_non_factual,corrector_is_factual,corrector_has_unknown,corrector_has_failed,corrector_n_extrinsic
26398483,Police have released an E-fit image of a man t...,False,True,False,False,1,Police have released an image of a man they wa...,True,False,False,False,1,Police have released an image of a man they wa...,True,False,False,False,1
30657223,Hartlepool United have signed Reading midfield...,True,False,False,False,6,Hartlepool United have signed Reading midfield...,True,False,False,False,5,Hartlepool United have signed Stags midfielder...,True,False,False,False,5
31113635,The amount of money spent on film production i...,False,True,False,False,2,The amount of money spent on film production i...,False,True,False,False,2,The amount of money spent on film production i...,False,True,False,False,2
32557344,The RSPCA has appealed for help to find the ow...,False,True,False,False,1,The RSPCA has appealed for information after a...,False,True,False,False,1,The RSPCA has appealed for information after a...,False,True,False,False,1
32798693,Retailer Next has lost a High Court battle wit...,True,False,False,False,1,The High Court has ruled in favour of HM Reven...,True,False,False,False,1,The High Court has ruled in favour of HM Reven...,True,False,False,False,1


In [6]:
# Helper method
def print_summaries(df, models):
    for sum_id in df.index:
        print(f"---XSUM ID {sum_id}---")
        row = df.loc[sum_id]
        for model in models:
            n_ents = row[model + '_n_extrinsic']
            summary = row[model + '_summary']
            print(f"{model} ({n_ents} extrinsic ents): {[summary]}")
        print()

### Summaries corrected by Oracle

In [7]:
df_inspect = df_sums[
    df_sums.fbs_oracle_is_factual
    & df_sums.baseline_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_oracle"
    ]
)

4
---XSUM ID 26398483---
baseline (1 extrinsic ents): ['Police have released an image of a man they want to speak to in connection with a sexual assault on a 15-year-old girl in Hertfordshire.']
fbs_oracle (1 extrinsic ents): ['Police have released an E-fit image of a man they want to speak to in connection with a sexual assault on a 14-year-old girl in Hertfordshire.']

---XSUM ID 36341722---
baseline (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £200,000.']
fbs_oracle (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £100,000.']

---XSUM ID 38470818---
baseline (3 extrinsic ents): ["St Johnstone's Gary Anderson says Rangers' John Kiernan should face a Scottish FA disciplinary hearing over an alleged punch."]
fbs_oracle (2 extrinsic ents): ["St Johnstone's Steven Anderson has said Rangers defender Kiernan should face a Scottish FA disciplinary hearing over an alleged pun

### Summaries messed by FBS Classifier

In [9]:
df_inspect = df_sums[
    df_sums.baseline_is_factual
    & df_sums.fbs_classifier_is_non_factual
]

print(df_inspect.shape)
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_classifier"
    ]
)

AttributeError: 'DataFrame' object has no attribute 'fbs_classifier_is_non_factual'

### Corrected by FBS Classifier

In [10]:
df_inspect = df_sums[
    df_sums.baseline_is_non_factual
    & df_sums.fbs_classifier_is_factual
]

print(df_inspect.shape)
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_classifier"
    ]
)

AttributeError: 'DataFrame' object has no attribute 'fbs_classifier_is_factual'

### Summaries messed up by Corrector

In [12]:
df_inspect = df_sums[
    df_sums.baseline_is_factual
    & df_sums.corrector_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "corrector"
    ]
)

1
---XSUM ID 40498102---
baseline (1 extrinsic ents): ["An exhibition in Iran of cartoonists' depictions of Donald Trump has drawn comparisons with Adolf Hitler."]
corrector (1 extrinsic ents): ['An exhibition in Tehran of cartoons of Trump has attracted more than 2,000 entries, many of which have drawn comparisons to Adolf Hitler.']

